#### Data Dictionary

* Survival	0 = No, 1 = Yes
* pclass =>	Ticket class (1 = 1st, 2 = 2nd, 3 = 3rd)
* sex = Sex
* Age => Age in years
* sibsp => # of siblings / spouses aboard the Titanic
* parch => # of parents / children aboard the Titanic
* ticket => Ticket number	
* fare => Passenger fare	
* cabin => Cabin number
* embarked => Port of Embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)



#### Variable Notes
#### pclass: A proxy for socio-economic status (SES)
* 1st = Upper
* 2nd = Middle
* 3rd = Lower



#### age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5



#### sibsp: The dataset defines family relations in this way...
* Sibling = brother, sister, stepbrother, stepsister
* Spouse = husband, wife (mistresses and fiancés were ignored)



#### parch: The dataset defines family relations in this way...
* Parent = mother, father
* Child = daughter, son, stepdaughter, stepson
* Some children travelled only with a nanny, therefore parch=0 for them.




In [ ]:
# read csv files

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# -- Feature Engineering --

In [ ]:
df = pd.read_csv('../input/train.csv')

df.head()

In [ ]:
test_df = pd.read_csv('../input/test.csv')
test_df.head()

In [ ]:
surv_col = df.iloc[:,1]
surv_col.head()

In [ ]:
train_df = df.iloc[:,2:]
train_df.head()

In [ ]:
pessengerId = test_df.iloc[:,0]
pessengerId.head()

In [ ]:
test_df.drop(['PassengerId'],axis=1,inplace=True)

test_df.head()

In [ ]:
train_df.head()

In [ ]:
concated_df = pd.concat([train_df,test_df])

concated_df.head()

In [ ]:
concated_df.info()

In [ ]:
from sklearn import preprocessing as prep

# Label Encoding for Sex Column

In [ ]:
le = prep.LabelEncoder()

concated_df.Sex =le.fit_transform(concated_df.Sex)

df.Sex[0:10]

In [ ]:
concated_df.head()

In [ ]:
concated_df.info()

### On the some columns, there are some missing values. Firstly I need to fill that columns

In [ ]:
embarked = concated_df['Embarked'].fillna('0')

embarked.unique()

# Label Encoding for Embarked Column

In [ ]:
concated_df.Embarked = le.fit_transform(embarked)

concated_df.Embarked.unique()

In [ ]:
concated_df.head()

In [ ]:
concated_df.tail()

In [ ]:
concated_df.dtypes

In [ ]:
print( 'Pclass:' ,concated_df.Pclass.unique())
print( 'Sex:' ,concated_df.Sex.unique())
print( 'SibSp:' ,concated_df.SibSp.unique())
print( 'Parch:' ,concated_df.Parch.unique())
print( 'Embarked:' ,concated_df.Embarked.unique())

### I will remove Cabin columns

In [ ]:
concated_df.drop(['Cabin'],axis=1,inplace=True)

concated_df.head()

In [ ]:
NameSplit = concated_df.Name.str.split('[,.]')

NameSplit.head()

In [ ]:
titles = [str.strip(name[1]) for name in NameSplit.values]
titles[:10]

In [ ]:
# new feature

concated_df['Title'] = titles

concated_df.head()

In [ ]:
concated_df.Title.unique()

In [ ]:
# useless words: I will combine Mademoiselle and Madame into a single type

concated_df.Title.values[concated_df.Title.isin(['Mme', 'Mmle'])] = 'Mmle'

In [ ]:
# keep reducing

concated_df.Title.values[concated_df.Title.isin(['Capt', 'Don', 'Major', 'Sir'])] = 'Sir'
concated_df.Title.values[concated_df.Title.isin(['Dona', 'Lady', 'the Countess', 'Jonkheer'])] = 'Lady'

In [ ]:
concated_df.Title.unique()

In [ ]:
# label encode new feature too

concated_df.Title = le.fit_transform(concated_df.Title)
concated_df.head()

In [ ]:
# new feature is family size
# number of spouses and siblings and oneself is family size

concated_df['FamilySize'] = concated_df.SibSp.values + concated_df.Parch.values + 1

In [ ]:
concated_df.head()

In [ ]:
surnames = [str.strip(name[0]) for name in NameSplit]
surnames[:10]

In [ ]:
concated_df['Surname'] = surnames
concated_df['FamilyID'] = concated_df.Surname.str.cat(concated_df.FamilySize.astype(str),sep='')
concated_df.head()

In [ ]:
# I will mark if any family id as small if family size is less than or equal to 2

concated_df.FamilyID.values[concated_df.FamilySize.values <= 2] = 'Small'

concated_df.head()

In [ ]:
# check up the frequency of family ids
concated_df.FamilyID.value_counts()

## There are too many family ids with few family members. maybe some families had different last names. I'll clean this.

In [ ]:
freq = list(dict(zip(concated_df.FamilyID.value_counts().index.tolist(), concated_df.FamilyID.value_counts().values)).items())

type(freq)

In [ ]:
freq = np.array(freq)

freq[:10]

In [ ]:
freq.shape

In [ ]:
# select the family ids with frequency of 2 or less
freq[freq[:,1].astype(int) <= 2].shape

In [ ]:
freq = freq[freq[:,1].astype(int) <= 2]

In [ ]:
# I'll assign 'Small' for those
concated_df.FamilyID.values[concated_df.FamilyID.isin(freq[:,0])] = 'Small'
concated_df.FamilyID.value_counts()

In [ ]:
# label encoding for family id

concated_df.FamilyID = le.fit_transform(concated_df.FamilyID)
concated_df.FamilyID.unique()

In [ ]:
# I will choose usefull features
concated_reduce = concated_df[[
    'Pclass', 'Sex', 'Age', 'SibSp',
    'Parch', 'Fare', 'Title', 'Embarked', 'FamilySize',
    'FamilyID']]

concated_reduce.head()

In [ ]:
concated_reduce.Age.unique()

In [ ]:
concated_reduce.info()

## There are missing values on Age Column. Therefore I will fill taking Median

In [ ]:
concated_reduce['Age'].fillna(concated_reduce['Age'].median(), inplace=True)
concated_reduce['Fare'].fillna(concated_reduce['Fare'].median(), inplace=True)

In [ ]:
concated_reduce.info()

# So, That Dataset (concated_reduce) is ready for spliting as Train and Test values.

In [ ]:
train_final = concated_reduce.iloc[:891].copy()
test_final = concated_reduce.iloc[891:].copy()

In [ ]:
train_final.head()

In [ ]:
test_final.head()

## At the outset, I splited some columns on the dataset. Now I will use that columns for creating Train dataset

In [ ]:
X = train_final.values

X

In [ ]:
y = surv_col.values

y

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
test_data = test_final.values

test_data

# Creating Neural Network with Keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [ ]:
model = Sequential()

model.add(Dense(32, init = 'uniform', activation='relu', input_dim = 10))
model.add(Dense(64, init = 'uniform', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, init = 'uniform', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(12, init = 'uniform', activation='relu'))
model.add(Dense(1, init = 'uniform', activation='sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(X,y, epochs=500, batch_size = 64, verbose = 1)

In [ ]:
pred = model.predict(test_data)

In [ ]:
pred

In [ ]:
# convert to integer
outputBin = np.zeros(0)
for i in pred:
    
    if i <= .5:
        
        outputBin = np.append(outputBin, 0)
    else:
        
        outputBin = np.append(outputBin, 1)
output = np.array(outputBin).astype(int)

In [ ]:
output

In [ ]:
d = {'PassengerId':pessengerId, 'Survived':output}

In [ ]:
final_df = pd.DataFrame(data=d)

In [ ]:
final_df.head()

In [ ]:
final = final_df.to_csv('new_result.csv',index=False) #convert to csv file

final

# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=350, max_depth=15, random_state=42)

print("train accuracy: {} ".format(rf.fit(X, y).score(X, y)))


In [ ]:
rf_pred = rf.predict(test_data)

In [ ]:
rf_pred

In [ ]:
r = {'PassengerId':pessengerId, 'Survived':rf_pred}

In [ ]:
final_rf = pd.DataFrame(data=r)

In [ ]:
final_rf.head(13)

In [ ]:
final_rf = final_df.to_csv('random_forest_result.csv',index=False) #convert to csv file

final_rf